## Проект "Анализ веб-документов" Техносфера Весна 2021

In [39]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

### Посмотрим на наши трейновые данные

In [40]:
traindf = pd.read_csv('data/train_groups.csv', sep=',', index_col='pair_id')
print(traindf.shape)
traindf.head(5)

(11690, 3)


,group_id,doc_id,target
pair_id,,,
1,1,15731,0
2,1,14829,0
3,1,15764,0
4,1,17669,0
5,1,14852,0


In [41]:
traindf.shape

(11690, 3)

# Идея 0 

Посмотрим на каждые заголовки, построим множество признаков, основанных на количестве схожих слов в заголовках.
Будем брать топ 15 в списке

## Решение 0
Повторим решение со второй домашней работы

1. Прочитаем все заголовки

In [42]:
doc_to_title = {}

with open('data/docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_to_title[int(data[0])] = '' if len(data) == 1 else data[1]

titlesdf = pd.DataFrame.from_dict(doc_to_title, orient='index', columns=['title'])
titlesdf.head(4)

,title
15731,ВАЗ 21213 | Замена подшипников ступицы | Нива
14829,"Ваз 2107 оптом в Сочи. Сравнить цены, купить п..."
15764,Купить ступица Лада калина2. Трансмиссия - пер...
17669,Классика 21010 - 21074


2. Прочитаем трейн и добавим заголовки к нему

In [43]:
def add_titles(df):
    df['title'] = titlesdf.loc[df['doc_id'].values].values

traindf = pd.read_csv('data/train_groups.csv', index_col='pair_id')
add_titles(traindf)
traindf.head(4)

,group_id,doc_id,target,title
pair_id,,,,
1,1,15731,0,ВАЗ 21213 | Замена подшипников ступицы | Нива
2,1,14829,0,"Ваз 2107 оптом в Сочи. Сравнить цены, купить п..."
3,1,15764,0,Купить ступица Лада калина2. Трансмиссия - пер...
4,1,17669,0,Классика 21010 - 21074


3. Найдем признаки. Посчитаем общие слова для всех групп и всех веб-страниц в заголовке. Как признаки для веб-страницы, возьмем значения топ-15  пересечений с другими страницами из группы. 

In [44]:
X_train, y_train, groups = [], [], []

In [45]:
def collect_titles_for_train(docs):
    titles = docs.title
    y_train.extend(docs.target.to_list())
    groups.extend(docs.group_id.to_list())
    for i, title in enumerate(titles):
        words = set(title.strip().split())
        distances = []
        for j, another_title in enumerate(titles):
            if i == j:
                continue
            another_words = set(another_title.strip().split())
            distances.append(len(words.intersection(another_words)))
        X_train.append(-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])

In [46]:
traindf.groupby('group_id').apply(collect_titles_for_train);
X, y, groups = np.asarray(X_train), np.asarray(y_train), np.asarray(groups)

In [47]:
from sklearn.preprocessing import StandardScaler

In [48]:
scaler = StandardScaler().fit(X, y)
X = scaler.transform(X);

In [49]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold

In [50]:
opt_params = {
    'n_jobs': -1
}

In [51]:
def ValScore(n_splits=5, groups=groups, *args, **kwargs):
    clf = SGDClassifier(*args, **kwargs)
    kf = GroupKFold(n_splits=n_splits)

    scores = []
    for train, test in kf.split(X, y, groups=groups):
        clf.fit(X[train], y[train])
        scores.append(f1_score(y_pred=clf.predict(X[test]),
                               y_true=y[test]))
    return np.asarray(scores)

In [52]:
def FindParams(param_name, param_range, known_params=opt_params):
    mean_scores = []

    for param in param_range:
        kwargs = known_params
        kwargs.update({param_name: param})
        scores = ValScore(**kwargs)
        mean_scores.append(scores.mean())

    opt_param = param_range[np.argmax(mean_scores)]
    return opt_param

In [53]:
grid = {
    'max_iter': [500, 1000, 2000, 3000, 5000],
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'alpha': np.logspace(4, -4, 10)
}

for param in grid:
    param_range = grid[param]
    opt_model_type = FindParams(param, param_range, opt_params)
    opt_params.update({param: opt_model_type})

/home/andrew/miniconda3/envs/da/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [54]:
opt_params

{'n_jobs': -1,
 'max_iter': 2000,
 'loss': 'squared_hinge',
 'alpha': 0.0007742636826811277}

### Считываем test и делаем predict

In [55]:
testdf = pd.read_csv('data/test_groups.csv', index_col='pair_id')
add_titles(testdf)
testdf.head(4)

,group_id,doc_id,title
pair_id,,,
11691,130,6710,КАК ПРОПИСАТЬ АДМИНКУ В КС 1.6 СЕБЕ ИЛИ ДРУГУ ...
11692,130,4030,Скачать: SGL-RP доработка | Слив мода [MySQL] ...
11693,130,5561,Как прописать админку в кс 1.6 - Counter-Strik...
11694,130,4055,Как прописать простую админку в кс 1 6


In [56]:
X_test, pairs_id = [], []

In [57]:
def collect_titles_for_test(docs):
    titles = docs.title
    pairs_id.extend(docs.index.to_list())
    for i, title in enumerate(titles):
        words = set(title.strip().split())
        distances = []
        for j, another_title in enumerate(titles):
            if i == j:
                continue
            another_words = set(another_title.strip().split())
            distances.append(len(words.intersection(another_words)))
        X_test.append(-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])

In [58]:
testdf.groupby('group_id').apply(collect_titles_for_test);
X_test = np.asarray(X_test)

In [59]:
X_test = scaler.transform(X_test)

In [60]:
clf = SGDClassifier(**opt_params).fit(X, y)
testdf['target'] = clf.predict(X_test)

In [61]:
testdf.drop(columns=['group_id', 'doc_id', 'title']).to_csv('solution0.csv')

### Итоговый скор 0.49019
Надо что-то получше

# Идея 1

Посмотрим на слова в получившихся данных.

In [62]:
from collections import Counter

In [63]:
wordCounter = Counter()
for title in titlesdf['title'].values:
    wordCounter.update(title.strip().split())
wordCounter.most_common(5)

[('-', 11915), ('в', 6517), ('и', 5654), ('|', 4754), ('на', 4269)]

Явно среди заголовков оказалось куча мусора и бесполезной информации.
Давайте предобработаем заголовки.

Из библиотеки nltk будем использовать токенайзер и стопслова. Из pymorphy2 возьмем морфемный анализ, а с помощью re избавимся от знаков препинания и тому подобного. С помощью string уберем пунктуацию

In [64]:
from nltk import corpus, word_tokenize
from string import digits, punctuation
from pymorphy2 import MorphAnalyzer

In [65]:
chars_to_remove = digits + punctuation + "–—«»"
morph = MorphAnalyzer()
RU_stopwords = set(corpus.stopwords.words('russian'))
EN_stopwords = set(corpus.stopwords.words('english'))
stopwords = RU_stopwords.union(EN_stopwords)

In [66]:
def process_title(title):
    title = bytes(title, 'utf-8').decode('utf-8', 'ignore')
    title = title.translate(str.maketrans('', '', chars_to_remove))
    words = word_tokenize(title)
    words = {word for word in words if word not in stopwords}
    words = {morph.parse(word)[0].normal_form for word in words}
    return words

In [67]:
titlesdf['title_words'] = [process_title(title) for title in titlesdf['title']]

Проверим как изменились наши заголовки

In [68]:
new_wordCounter = Counter()
for words in titlesdf['title_words'].values:
    new_wordCounter.update(words)
new_wordCounter.most_common(5)

[('как', 3914),
 ('форум', 1654),
 ('скачать', 1020),
 ('страница', 1019),
 ('онлайн', 848)]

Уже лучше, попробуем запустить решение 0 на этих данных

## Решение 1

In [69]:
def add_title_words(df):
    df['title_words'] = [words for words in titlesdf.loc[df['doc_id'].values]['title_words']]

In [70]:
add_title_words(traindf)
traindf

,group_id,doc_id,target,title,title_words
pair_id,,,,,
1,1,15731,0,ВАЗ 21213 | Замена подшипников ступицы | Нива,"{ступица, ваз, замена, подшипник, нива}"
2,1,14829,0,"Ваз 2107 оптом в Сочи. Сравнить цены, купить п...","{товар, оптом, сравнить, сочи, ваз, tiuru, цен..."
3,1,15764,0,Купить ступица Лада калина2. Трансмиссия - пер...,"{ступица, калин, лада, трансмиссия, замена, пе..."
4,1,17669,0,Классика 21010 - 21074,{классика}
5,1,14852,0,Ступица Нива — замена подшипника своими руками,"{ступица, рука, нива, замена, подшипник, свой}"
...,...,...,...,...,...
11686,129,26672,0,❤★✿★АПРЕЛЯТА 2014 -6❤★✿,"{❤★✿, ❤★✿★апрелёнок}"
11687,129,25838,0,:: Gästebuch,{gästebuch}
11688,129,25703,0,Jizolofej: Archive,"{jizolofej, archive}"


In [71]:
X_train, y_train, groups = [], [], []

In [72]:
def collect_title_words_for_train(docs):
    title_words = docs.title_words
    y_train.extend(docs.target.to_list())
    groups.extend(docs.group_id.to_list())
    for i, words in enumerate(title_words):
        distances = []
        for j, another_words in enumerate(title_words):
            if i == j:
                continue
            distances.append(len(words.intersection(another_words)))
        X_train.append(-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])

In [73]:
traindf.groupby('group_id').apply(collect_title_words_for_train);
X, y, groups = np.asarray(X_train), np.asarray(y_train), np.asarray(groups)

In [74]:
scaler = StandardScaler().fit(X, y)
X = scaler.transform(X);

In [75]:
opt_params = {
    'n_jobs': -1
}

In [76]:
grid = {
    'max_iter': [500, 1000, 2000, 3000, 5000],
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'alpha': np.logspace(4, -4, 10)
}

for param in grid:
    param_range = grid[param]
    opt_model_type = FindParams(param, param_range, opt_params)
    opt_params.update({param: opt_model_type})

In [77]:
opt_params

{'n_jobs': -1, 'max_iter': 5000, 'loss': 'log', 'alpha': 0.04641588833612782}

### Считываем test и делаем predict

In [78]:
X_test = []

In [79]:
def collect_title_words_for_test(docs):
    title_words = docs.title_words
    for i, words in enumerate(title_words):
        distances = []
        for j, another_words in enumerate(title_words):
            if i == j:
                continue
            distances.append(len(words.intersection(another_words)))
        X_test.append(-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])

In [80]:
testdf = pd.read_csv('data/test_groups.csv', index_col='pair_id')
add_title_words(testdf)
testdf.head(4)

,group_id,doc_id,title_words
pair_id,,,
11691,130,6710,"{youtube, или, как, прописать, друг, себя, кс,..."
11692,130,4030,"{доработка, e, gta, скачать, mysql, samp, мода..."
11693,130,5561,"{каталог, как, прописать, игровой, портал, cou..."
11694,130,4055,"{как, прописать, кс, админк, простой}"


In [81]:
testdf.groupby('group_id').apply(collect_title_words_for_test);
X_test = np.asarray(X_test)

In [82]:
X_test = scaler.transform(X_test)

In [83]:
clf = SGDClassifier(**opt_params).fit(X, y)
testdf['target'] = clf.predict(X_test)

In [84]:
testdf.drop(columns=['group_id', 'doc_id', 'title_words']).to_csv('solution1.csv')

### Итоговый скор 0.63710
Ура, побили бейзлайн!

# Идея 2

Посмотрим на наши файлы

In [86]:
from bs4 import BeautifulSoup
import codecs
with codecs.open('content/1.dat', 'r', 'utf-8') as f:
    url = f.readline().strip()
    soup = 

М. Б. Аншина Центр репродукции и генетики «ФертиМед», г. Москва


## Решение 2